# Active Learning

In [1]:
import pandas as pd
import numpy as np

## Load Data

## Annotate Data

## Train Model

## Model Accuracy

## Get Low Predictions to Annotate

## Get High Predictions

## Update Labelled Data

## Retrain Model

## Model Accuracy

## Concluding Remarks

---